In [1]:

####Ploting Histogram

import pandas as pd
import numpy as np

from bokeh.plotting import figure
from bokeh.models import (CategoricalColorMapper, HoverTool, 
						  ColumnDataSource, Panel, 
						  FuncTickFormatter, SingleIntervalTicker, LinearAxis)
from bokeh.models.widgets import (CheckboxGroup, Slider, RangeSlider, 
								  Tabs, CheckboxButtonGroup, 
								  TableColumn, DataTable, Select)
from bokeh.layouts import column, row, WidgetBox
from bokeh.palettes import Category20_16

# Make plot with histogram and return tab
def histogram_tab(rentals):

	# Function to make a dataset for histogram based on a list of carriers
	# a minimum delay, maximum delay, and histogram bin width
	def make_dataset(property_list, range_start =-1, range_end = 10, bin_width = 0.5):

		# Dataframe to hold information
		by_property = pd.DataFrame(columns=['proportion', 'left', 'right', 
										   'f_proportion', 'f_interval',
										   'name', 'color'])
		
		range_extent = range_end - range_start

		# Iterate through all the carriers
		for i, property_name in enumerate(property_list):

			# Subset to the carrier
			subset = rentals[rentals['PropertyType'] == property_name]

			# Create a histogram with 5 minute bins
			arr_hist, edges = np.histogram(subset['VacancyRate'], 
										   bins = int(range_extent / bin_width), 
										   range = [range_start, range_end])

			# Divide the counts by the total to get a proportion
			arr_df = pd.DataFrame({'proportion': arr_hist / np.sum(arr_hist), 'left': edges[:-1], 'right': edges[1:] })

			# Format the proportion 
			arr_df['f_proportion'] = ['%0.5f' % proportion for proportion in arr_df['proportion']]

			# Format the interval
			arr_df['f_interval'] = ['%d to %d percents' % (left, right) for left, right in zip(arr_df['left'], arr_df['right'])]

			# Assign the carrier for labels
			arr_df['name'] = property_name

			# Color each carrier differently
			arr_df['color'] = Category20_16[i]

			# Add to the overall dataframe
			by_property = by_property.append(arr_df)

		# Overall dataframe
		by_property = by_property.sort_values(['name', 'left'])

		return ColumnDataSource(by_property)

	def style(p):
		# Title 
		p.title.align = 'center'
		p.title.text_font_size = '20pt'
		p.title.text_font = 'serif'

		# Axis titles
		p.xaxis.axis_label_text_font_size = '14pt'
		p.xaxis.axis_label_text_font_style = 'bold'
		p.yaxis.axis_label_text_font_size = '14pt'
		p.yaxis.axis_label_text_font_style = 'bold'

		# Tick labels
		p.xaxis.major_label_text_font_size = '12pt'
		p.yaxis.major_label_text_font_size = '12pt'

		return p
	
	def make_plot(src):
		# Blank plot with correct labels
		p = figure(plot_width = 700, plot_height = 700, 
				  title = 'Histogram of Vacancy Rate by Property Type',
				  x_axis_label = 'Vacancy Rate (percent)', y_axis_label = 'Proportion')

		# Quad glyphs to create a histogram
		p.quad(source = src, bottom = 0, top = 'proportion', left = 'left', right = 'right',
			   color = 'color', fill_alpha = 0.7, hover_fill_color = 'color', legend = 'name',
			   hover_fill_alpha = 1.0, line_color = 'black')

		# Hover tool with vline mode
		hover = HoverTool(tooltips=[('PropertyType', '@name'), 
									('VacancyRate', '@f_interval'),
									('Proportion', '@f_proportion')],
						  mode='vline')

		p.add_tools(hover)

		# Styling
		p = style(p)

		return p
	
	
	
	def update(attr, old, new):
		property_to_plot = [property_selection.labels[i] for i in property_selection.active]
		
		new_src = make_dataset(property_to_plot,
							   range_start = range_select.value[0],
							   range_end = range_select.value[1],
							   bin_width = binwidth_select.value)
		
		

		src.data.update(new_src.data)
		
	# Carriers and colors
	available_property= list(set(rentals['PropertyType']))
	available_property.sort()


	property_colors = Category20_16
	property_colors.sort()
		
	property_selection = CheckboxGroup(labels=available_property, 
									  active = [0, 1,2,3,4])
	property_selection.on_change('active', update)
	
	binwidth_select = Slider(start = 1, end = 10, 
							 step = 0.5, value = 1,
							 title = 'Bin Width (min)')
	binwidth_select.on_change('value', update)
	
	range_select = RangeSlider(start = 0, end = 10, value = (0, 10),
							   step = 0.5, title = 'Range of Vacancy Rate')
	range_select.on_change('value', update)
	
	# Initial carriers and data source
	initial_property = [property_selection.labels[i] for i in property_selection.active]
	
	src = make_dataset(initial_property,
					   range_start = range_select.value[0],
					   range_end = range_select.value[1],
					   bin_width = binwidth_select.value)
	p = make_plot(src)
	
	# Put controls in a single element
	controls = WidgetBox(property_selection, binwidth_select, range_select)
	
	# Create a row layout
	layout = row(controls, p)
	
	# Make a tab with the layout 
	tab = Panel(child=layout, title = 'Histogram')

	return tab

In [2]:
###Plotting Density
# pandas and numpy for data manipulation
import pandas as pd
import numpy as np

from scipy.stats import gaussian_kde

from bokeh.plotting import figure
from bokeh.models import (CategoricalColorMapper, HoverTool, 
						  ColumnDataSource, Panel, 
						  FuncTickFormatter, SingleIntervalTicker, LinearAxis)
from bokeh.models.widgets import (CheckboxGroup, Slider, RangeSlider, 
								  Tabs, CheckboxButtonGroup, 
								  TableColumn, DataTable, Select)
from bokeh.layouts import column, row, WidgetBox
from bokeh.palettes import Category20_16

def density_tab(rentals):
	
	# Dataset for density plot based on carriers, range of delays,
	# and bandwidth for density estimation
	def make_dataset(Property_list, range_start, range_end, bandwidth):

		xs = []
		ys = []
		colors = []
		labels = []

		for i, Property in enumerate(Property_list):
			subset = rentals[rentals['PropertyType'] == Property]
			subset = subset[subset['AverageRent'].between(range_start, 
														range_end)]

			kde = gaussian_kde(subset['AverageRent'], bw_method=bandwidth)
			
			# Evenly space x values
			x = np.linspace(range_start, range_end, 100)
			# Evaluate pdf at every value of x
			y = kde.pdf(x)

			# Append the values to plot
			xs.append(list(x))
			ys.append(list(y))

			# Append the colors and label
			colors.append(property_colors[i])
			labels.append(Property)

		new_src = ColumnDataSource(data={'x': xs, 'y': ys, 
								   'color': colors, 'label': labels})

		return new_src

	def make_plot(src):
		p = figure(plot_width = 700, plot_height = 700,
				   title = 'Density Plot of Average Rent by Property Type',
				   x_axis_label = 'Price (average)', y_axis_label = 'Density')


		p.multi_line('x', 'y', color = 'color', legend = 'label', 
					 line_width = 3,
					 source = src)

		# Hover tool with next line policy
		hover = HoverTool(tooltips=[('PropertyType', '@label'), 
									('AverageRent', '$x'),
									('Density', '$y')],
						  line_policy = 'next')

		# Add the hover tool and styling
		p.add_tools(hover)

		p = style(p)

		return p
	
	def update(attr, old, new):
		# List of carriers to plot
		property_to_plot = [property_selection.labels[i] for i in 
							property_selection.active]
		
		# If no bandwidth is selected, use the default value
		if bandwidth_choose.active == []:
			bandwidth = None
		# If the bandwidth select is activated, use the specified bandwith
		else:
			bandwidth = bandwidth_select.value
			
		
		new_src = make_dataset(property_to_plot,
									range_start = range_select.value[0],
									range_end = range_select.value[1],
									bandwidth = bandwidth)
		
		src.data.update(new_src.data)
		
	def style(p):
		# Title 
		p.title.align = 'center'
		p.title.text_font_size = '20pt'
		p.title.text_font = 'serif'

		# Axis titles
		p.xaxis.axis_label_text_font_size = '14pt'
		p.xaxis.axis_label_text_font_style = 'bold'
		p.yaxis.axis_label_text_font_size = '14pt'
		p.yaxis.axis_label_text_font_style = 'bold'

		# Tick labels
		p.xaxis.major_label_text_font_size = '12pt'
		p.yaxis.major_label_text_font_size = '12pt'

		return p
	
	# Carriers and colors
	available_property = list(set(rentals['PropertyType']))
	available_property.sort()

	property_colors = Category20_16
	property_colors.sort()

	# Carriers to plot
	property_selection = CheckboxGroup(labels=available_property, 
									   active = [0,1,2,3,4])
	property_selection.on_change('active', update)
	
	range_select = RangeSlider(start = 0, end = 2000, value = (0, 2000),
							   step = 100, title = 'Range of Rent (average)')
	range_select.on_change('value', update)
	
	# Initial carriers and data source
	initial_property = [property_selection.labels[i] for 
						i in property_selection.active]
	
	# Bandwidth of kernel
	bandwidth_select = Slider(start = 0.1, end = 50, 
							  step = 5, value = 0.5,
							  title = 'Bandwidth for Density Plot')
	bandwidth_select.on_change('value', update)
	
	# Whether to set the bandwidth or have it done automatically
	bandwidth_choose = CheckboxButtonGroup(
		labels=['Choose Bandwidth (Else Auto)'], active = [])
	bandwidth_choose.on_change('active', update)

	# Make the density data source
	src = make_dataset(initial_property, 
						range_start = range_select.value[0],
						range_end = range_select.value[1],
						bandwidth = bandwidth_select.value) 
	
	# Make the density plot
	p = make_plot(src)
	
	# Add style to the plot
	p = style(p)
	
	# Put controls in a single element
	controls = WidgetBox(property_selection, range_select, 
						 bandwidth_select, bandwidth_choose)
	
	# Create a row layout
	layout = row(controls, p)
	
	# Make a tab with the layout 
	tab = Panel(child=layout, title = 'Density Plot')

	return tab

In [3]:
# Pandas for data management
import pandas as pd
import numpy as np
from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application
from bokeh.models import RadioGroup, ColumnDataSource, WidgetBox
from bokeh.layouts import row
import numpy as np
import holoviews as hv
import pandas as pd
import seaborn as sns
import matplotlib
import datetime
from matplotlib.dates import DateFormatter
from matplotlib.dates import HourLocator
matplotlib.use('agg')
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'


# Import Multiple CSVs and combining them
from bokeh.io import show, curdoc
from bokeh.io import output_file, show
from bokeh.layouts import layout
from bokeh.models import Slider, Button
from bokeh.plotting import figure
from bokeh.models import HoverTool, ColumnDataSource
from bokeh.io import curdoc
from bokeh.models import CategoricalColorMapper
from bokeh.palettes import Spectral6
from bokeh.layouts import row, widgetbox
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import Select
from bokeh.plotting import output_notebook, show
renderer = hv.renderer('bokeh').instance(mode='server')

# os methods for manipulating paths
from os.path import dirname, join

# Bokeh basics 
from bokeh.io import curdoc
from bokeh.models.widgets import Tabs

# Using included state data from Bokeh for map
from bokeh.sampledata.us_states import data as states

# Read data into dataframes
rentals = pd.read_csv(("C:/Users/Lenovo/Desktop/Rental/Permanent Rental/By Times/Primary Rental Comparision(Apartment)/Total Apartment Toronto_cleaned.csv"), 
	                                          index_col=0).dropna()
rentals[rentals == 'NA'] = np.nan
# fill missing values with mean column values
rentals.fillna(rentals.mean(), inplace=True)
#print(rentals)
from bokeh.models import ColumnDataSource
df=ColumnDataSource(rentals)
from bokeh.models import HoverTool
output_notebook()

def modify_doc(doc):
    # Make the ColumnDataSource: source
    # Create each of the tabs
    tab1 = histogram_tab(rentals)
    tab2 = density_tab(rentals)
    #tab4 = route_tab(rentalarea)
    # Put all the tabs into one application
    tabs = Tabs(tabs =[tab1,tab2])
    doc.add_root(tabs)
handler = FunctionHandler(modify_doc)
app = Application(handler)
show(app)

Loading BokehJS ...

INFO:bokeh.server.server:Starting Bokeh server version 1.0.2 (running on Tornado 5.1.1)


INFO:tornado.access:200 GET /autoload.js?bokeh-autoload-element=1003&bokeh-absolute-url=http://localhost:49688&resources=none (::1) 124.80ms
INFO:tornado.access:101 GET /ws?bokeh-protocol-version=1.0&bokeh-session-id=itawx5GAd4kUVyDceOSFecu189qjc3NQhR9AViaE53u5 (::1) 0.00ms
INFO:bokeh.server.views.ws:WebSocket connection opened
INFO:bokeh.server.views.ws:ServerConnection created
